# Python 裝飾器（Decorator）用法筆記

## 1. 什麼是裝飾器？
- **裝飾器**是一種「高階函式」（higher-order function），接受一個函式作為參數，並返回一個新函式，用以在不改動原函式的前提下，給其添加額外行為。

## 2. 基本語法

In [1]:
def decorator(func):
    def wrapper(*args, **kwargs):
        # 裝飾前：可執行任意程式碼
        result = func(*args, **kwargs)
        # 裝飾後：可執行任意程式碼
        return result

    return wrapper


@decorator
def my_function(x, y):
    return x + y


# 等同於：
# my_function = decorator(my_function)
print(my_function(1, 2))

3


## 3. 範例：紀錄函式執行時間

In [2]:
import time


def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - start
        print(f"{func.__name__} 耗時：{elapsed:.4f} 秒")
        return result

    return wrapper


@timer
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)


print(fib(10))


fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0010 秒
fib 耗時：0.0000 秒
fib 耗時：0.0010 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0010 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0010 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0010 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0010 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0.0000 秒
fib 耗時：0

## 4. 帶參數的裝飾器
- 如果要讓裝飾器本身接受參數，需要再包一層函式：

In [3]:
def repeat(times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(times):
                func(*args, **kwargs)

        return wrapper

    return decorator


@repeat(times=3)
def greet(name):
    print(f"Hello, {name}!")


greet("Alice")  # 會印三次


Hello, Alice!
Hello, Alice!
Hello, Alice!


## 5. 堆疊多個裝飾器
- 多個 `@` 裝飾器時，從上到下依序套用（底層到上層）：

In [4]:
def decorator_a(func):
    def wrapper(*args, **kwargs):
        print("裝飾器 A 開始")
        result = func(*args, **kwargs)
        print("裝飾器 A 結束")
        return result

    return wrapper


def decorator_b(func):
    def wrapper(*args, **kwargs):
        print("裝飾器 B 開始")
        result = func(*args, **kwargs)
        print("裝飾器 B 結束")
        return result

    return wrapper


@decorator_a
@decorator_b
def func():
    print("run func()")


# 等同於：func = decorator_a(decorator_b(func))
func()

裝飾器 A 開始
裝飾器 B 開始
run func()
裝飾器 B 結束
裝飾器 A 結束


## 6. 使用 `functools.wraps`
- 為了保留原函式的名稱、文件字串（`__doc__`）等屬性，建議在 `wrapper` 上加上 `@functools.wraps(func)`：

In [5]:
import functools


def decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        """wrapper 的 docstring"""
        return func(*args, **kwargs)

    return wrapper


@decorator
def func():
    print("run func()")
    
func()

run func()


## 7. 類別裝飾器
- 裝飾類別時，裝飾器接收並返回一個類別：

In [6]:
def singleton(cls):
    instances = {}

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance


@singleton
class MyClass:
    pass


a = MyClass()
b = MyClass()
print(a is b)
assert a is b

True


## 8. 實用建議

1. 命名清晰：裝飾器本身與包裹函式都要有清楚命名，利於除錯。

2. 保留原函式元資料：使用 `@functools.wraps`。

3. 避免副作用：裝飾器應盡量不要修改輸入參數或全域狀態，除非刻意設計。

4. 測試覆蓋：為裝飾器與被裝飾函式各寫測試，確保功能正確。